# Лабораторная работа 3
## Генералов Даниил, 1032212280

> В данное лабораторной работе рассматривается Web API социальной сети Вконтакте. В отличие от API сервиса, рассмотренного в предыдущей лабораторной работе, API VK включает в себя несколько сотен методов, разные варианты авторизации, набор SDK под различные платформы, а также обширную документацию, некоторые части которой оформлены в виде пособий для новичков.
> 
> В ходе лабораторной работы, необходимо будет познакомиться с некоторыми разделами документации, самостоятельно, найти ряд методов интерфейса и сформировать запросы на получение тех или иных данных


> Исходный код этого файла доступен в git-репозитории: https://github.com/danya02/rudn-year3-api-labs/blob/main/lab3/main.ipynb

## Обзор API VK

Сайт Вконтакте использует псевдо-RPC API, где методы находятся на отдельных URL,
а параметры передаются через query-строку. 
В результате этого любые операции, которые требуют контента
(например, загрузка картинок)
создают URL, на который следует отправить этот контент другим способом.

Когда требуется вызвать метод, нужно сделать POST-запрос на https://api.vk.com/method/`название метода`?`параметры`.
Название метода состоит из названия группы и названия индивидуальной операции: например, `users.get` -- это операция `get` в категории `users`.
Чтобы узнать, что возвращает какой-то метод, можно прочитать документацию этого метода: https://dev.vk.com/ru/method/users.get

Ключ авторизации следует передавать в header как `Authorization: Bearer` (или как один из параметров `access_token`). 
(Почти) Все методы сейчас требуют какого-то ключа доступа, но он не обязан быть привязан к пользователю: можно использовать сервисный токен, который вместо этого привязан к приложению.

Существует небольшое количество методов, которые можно было вызвать без ключей доступа: например, те, которые в категории `database`.
Теперь они также требуют ключей доступа, но можно использовать сервисный ключ.

In [1]:
import getpass
import requests
SVC_TOKEN = getpass.getpass("Введите сервисный ключ приложения:")

Чтобы получить доступ к данным пользователя, нужно получить токен от него.
Самый простой способ это сделать -- через Implicit Flow,
который по сути является стандартным OAuth.

Этот способ доступа является legacy,
и нельзя создать новые приложения, которые используют его.
К счастью, у меня уже есть приложение,
которое я создал раньше этого изменения.

Для того, чтобы авторизоваться в нем,
нужно перейти на https://oauth.vk.com/authorize?client_id=6733081&redirect_uri=blank.html&scope=142558&response_type=token
и затем скопировав токен из адресной строки в поле ниже.

Также можно получить вечный доступ, добавив к scope 65536 -- это указывает требование доступа offline.

In [2]:
USER_TOKEN=getpass.getpass("Введите токен из адресной строки: access_token=")

In [3]:
import pandas
import pprint
import itertools
import time
from IPython.display import HTML, display


def print_transaction(resp: requests.Response):
    """Показать одно HTTP-взаимодействие красиво"""
    left = f"{resp.request.method} {resp.request.url.replace(SVC_TOKEN, '**SVC_TOKEN**').replace(USER_TOKEN, '**USER_TOKEN**')}\n"
    for k,v in resp.request.headers.items():
        left += f"{k}: {v}\n"

    right = f"{resp.status_code}\n"
    for k,v in resp.headers.items():
        right += f"{k}: {v}\n"
    right += "\n"
    right += pprint.pformat(resp.json())

    table = []
    for l,r in itertools.zip_longest(left.splitlines(), right.splitlines(), fillvalue=''):
        table.append([l,r])
    display(HTML('<style>table {text-align: left !important; font-family: monospace; white-space: pre;} .jp-OutputArea-child { overflow: scroll !important; }</style>'))
    df = pandas.DataFrame(table)
    time.sleep(1)
    return HTML(df.style.set_table_styles([{'selector': 'td', 'props': 'text-align: left !important; '}]).to_html())
    


## Методы

Теперь можно получать доступ к данным и выполнять действия. Например, найти пользователей по имени, где результаты будут упорядочены по релевантности: в частности, если пользователь, которому принадлежит токен, имеет это имя, то он будет выведен первым.

In [4]:
print_transaction(
    requests.post("https://api.vk.com/method/users.search",
                  params={
                      "v": "5.199",
                      "access_token": USER_TOKEN,
                      "q": "Генералов"
                  })
)

,0,1
0,POST https://api.vk.com/method/users.search?v=5.199&access_token=**USER_TOKEN**&q=%D0%93%D0%B5%D0%BD%D0%B5%D1%80%D0%B0%D0%BB%D0%BE%D0%B2,200
1,User-Agent: python-requests/2.31.0,Server: kittenx
2,"Accept-Encoding: gzip, deflate, br","Date: Sat, 20 Apr 2024 13:35:08 GMT"
3,Accept: */*,Content-Type: application/json; charset=utf-8
4,Connection: keep-alive,Content-Length: 1806
5,Content-Length: 0,Connection: keep-alive
6,,X-Powered-By: KPHP/7.4.116528
7,,"Set-Cookie: remixir=DELETED; expires=Thu, 01 Jan 1970 00:00:01 GMT; path=/; domain=.vk.com; secure; HttpOnly"
8,,Cache-control: no-store
9,,X-Frame-Options: DENY


Можно вывести меньше результатов, но зато получить больше информации о каждом из них, а также отфильтровать их по свойствам:

In [5]:
print_transaction(
    requests.post("https://api.vk.com/method/users.search",
                  params={
                      "v": "5.199",
                      "access_token": USER_TOKEN,
                      "q": "Генералов",
                      "fields": "about,activities,bdate,books,career,city,common_count,connections,contacts,country,domain,education,exports,followers_count,home_town,last_seen,military,movies,muisic,nickname,occupation,online,personal,photo_max,quotes,relation,sex,universities",
                      "count": 2,
                      "city": 1914764,
                  })
)

,0,1
0,POST https://api.vk.com/method/users.search?v=5.199&access_token=**USER_TOKEN**&q=%D0%93%D0%B5%D0%BD%D0%B5%D1%80%D0%B0%D0%BB%D0%BE%D0%B2&fields=about%2Cactivities%2Cbdate%2Cbooks%2Ccareer%2Ccity%2Ccommon_count%2Cconnections%2Ccontacts%2Ccountry%2Cdomain%2Ceducation%2Cexports%2Cfollowers_count%2Chome_town%2Clast_seen%2Cmilitary%2Cmovies%2Cmuisic%2Cnickname%2Coccupation%2Conline%2Cpersonal%2Cphoto_max%2Cquotes%2Crelation%2Csex%2Cuniversities&count=2&city=1914764,200
1,User-Agent: python-requests/2.31.0,Server: kittenx
2,"Accept-Encoding: gzip, deflate, br","Date: Sat, 20 Apr 2024 13:35:09 GMT"
3,Accept: */*,Content-Type: application/json; charset=utf-8
4,Connection: keep-alive,Content-Length: 988
5,Content-Length: 0,Connection: keep-alive
6,,X-Powered-By: KPHP/7.4.116528
7,,"Set-Cookie: remixir=DELETED; expires=Thu, 01 Jan 1970 00:00:01 GMT; path=/; domain=.vk.com; secure; HttpOnly"
8,,Cache-control: no-store
9,,X-Frame-Options: DENY


## Database
Категория методов `Database` позволяет узнавать список известных VK сущностей в реальном мире: стран, городов, школ и ВУЗов; а также находить их идентификаторы для использования в других запросах. 

In [6]:
print_transaction(
    requests.post("https://api.vk.com/method/database.getCountries",
                  params={
                      "access_token": SVC_TOKEN,
                      "v": "5.199",
                      "code": "RU"
                  })
)

,0,1
0,POST https://api.vk.com/method/database.getCountries?access_token=**SVC_TOKEN**&v=5.199&code=RU,200
1,User-Agent: python-requests/2.31.0,Server: kittenx
2,"Accept-Encoding: gzip, deflate, br","Date: Sat, 20 Apr 2024 13:35:10 GMT"
3,Accept: */*,Content-Type: application/json; charset=utf-8
4,Connection: keep-alive,Content-Length: 76
5,Content-Length: 0,Connection: keep-alive
6,,X-Powered-By: KPHP/7.4.116528
7,,"Set-Cookie: remixir=DELETED; expires=Thu, 01 Jan 1970 00:00:01 GMT; path=/; domain=.vk.com; secure; HttpOnly"
8,,Cache-control: no-store
9,,X-Frame-Options: DENY


In [7]:
print_transaction(
    requests.post("https://api.vk.com/method/database.getCities",
                  params={
                      "access_token": SVC_TOKEN,
                      "v": "5.199",
                      "q": "москва"
                  })
)

,0,1
0,POST https://api.vk.com/method/database.getCities?access_token=**SVC_TOKEN**&v=5.199&q=%D0%BC%D0%BE%D1%81%D0%BA%D0%B2%D0%B0,200
1,User-Agent: python-requests/2.31.0,Server: kittenx
2,"Accept-Encoding: gzip, deflate, br","Date: Sat, 20 Apr 2024 13:35:12 GMT"
3,Accept: */*,Content-Type: application/json; charset=utf-8
4,Connection: keep-alive,Content-Length: 328
5,Content-Length: 0,Connection: keep-alive
6,,X-Powered-By: KPHP/7.4.116528
7,,"Set-Cookie: remixir=DELETED; expires=Thu, 01 Jan 1970 00:00:01 GMT; path=/; domain=.vk.com; secure; HttpOnly"
8,,Cache-control: no-store
9,,X-Frame-Options: DENY


In [8]:
print_transaction(
    requests.post("https://api.vk.com/method/database.getSchools",
                  params={
                      "access_token": SVC_TOKEN,
                      "v": "5.199",
                      "city_id": 1,
                      "q": "им. баумана"
                  })
)

,0,1
0,POST https://api.vk.com/method/database.getSchools?access_token=**SVC_TOKEN**&v=5.199&city_id=1&q=%D0%B8%D0%BC.+%D0%B1%D0%B0%D1%83%D0%BC%D0%B0%D0%BD%D0%B0,200
1,User-Agent: python-requests/2.31.0,Server: kittenx
2,"Accept-Encoding: gzip, deflate, br","Date: Sat, 20 Apr 2024 13:35:13 GMT"
3,Accept: */*,Content-Type: application/json; charset=utf-8
4,Connection: keep-alive,Content-Length: 322
5,Content-Length: 0,Connection: keep-alive
6,,X-Powered-By: KPHP/7.4.116528
7,,"Set-Cookie: remixir=DELETED; expires=Thu, 01 Jan 1970 00:00:01 GMT; path=/; domain=.vk.com; secure; HttpOnly"
8,,Cache-control: no-store
9,,X-Frame-Options: DENY


In [9]:
print_transaction(
    requests.post("https://api.vk.com/method/database.getUniversities",
                  params={
                      "access_token": SVC_TOKEN,
                      "v": "5.199",
                      "q": "РУДН"
                  })
)

,0,1
0,POST https://api.vk.com/method/database.getUniversities?access_token=**SVC_TOKEN**&v=5.199&q=%D0%A0%D0%A3%D0%94%D0%9D,200
1,User-Agent: python-requests/2.31.0,Server: kittenx
2,"Accept-Encoding: gzip, deflate, br","Date: Sat, 20 Apr 2024 13:35:14 GMT"
3,Accept: */*,Content-Type: application/json; charset=utf-8
4,Connection: keep-alive,Content-Length: 912
5,Content-Length: 0,Connection: keep-alive
6,,X-Powered-By: KPHP/7.4.116528
7,,"Set-Cookie: remixir=DELETED; expires=Thu, 01 Jan 1970 00:00:01 GMT; path=/; domain=.vk.com; secure; HttpOnly"
8,,Cache-control: no-store
9,,X-Frame-Options: DENY


## Документы

Не совсем ясно, как работать с аудиозаписями: кажется эта фича больше не существует даже в веб-интерфейсе.

С документами можно работать с помощью методов `docs`.
Для доступа к ним требуется отдельный scope в токене доступа, который уже был запрошен выше.

Можно получить доступ к списку документов, которые принадлежат пользователю, отфильтровав его по типу и ограничив количество:

In [10]:
print_transaction(
    requests.post("https://api.vk.com/method/docs.get",
                  params={
                      "access_token": USER_TOKEN,
                      "v": "5.199",
                      "count": 5,
                      "type": 3, # gif
                  })
)

,0,1
0,POST https://api.vk.com/method/docs.get?access_token=**USER_TOKEN**&v=5.199&count=5&type=3,200
1,User-Agent: python-requests/2.31.0,Server: kittenx
2,"Accept-Encoding: gzip, deflate, br","Date: Sat, 20 Apr 2024 13:35:15 GMT"
3,Accept: */*,Content-Type: application/json; charset=utf-8
4,Connection: keep-alive,Content-Length: 1025
5,Content-Length: 0,Connection: keep-alive
6,,X-Powered-By: KPHP/7.4.116528
7,,"Set-Cookie: remixir=DELETED; expires=Thu, 01 Jan 1970 00:00:01 GMT; path=/; domain=.vk.com; secure; HttpOnly"
8,,Cache-control: no-store
9,,X-Frame-Options: DENY


Чтобы загрузить документ, требуется сначала выполнить один из методов `docs.get*UploadServer`,
затем отправить POST-запрос на этот адрес с содержимым.
В ответ сервер возвращает информацию,
которую нужно отправить в метод `docs.save`,
чтобы финализировать сохранение документа.

In [11]:
# Сделать документ
with open('/tmp/galtse.cx.txt', 'w') as o:
    o.write(requests.get('http://galtse.cx').text.replace('<br><br>', '\n'))

In [12]:
# Получить адрес для загрузки
resp = requests.post("https://api.vk.com/method/docs.getUploadServer",
                  params={
                      "access_token": USER_TOKEN,
                      "v": "5.199",
                  })

print_transaction(resp)

,0,1
0,POST https://api.vk.com/method/docs.getUploadServer?access_token=**USER_TOKEN**&v=5.199,200
1,User-Agent: python-requests/2.31.0,Server: kittenx
2,"Accept-Encoding: gzip, deflate, br","Date: Sat, 20 Apr 2024 13:35:18 GMT"
3,Accept: */*,Content-Type: application/json; charset=utf-8
4,Connection: keep-alive,Content-Length: 177
5,Content-Length: 0,Connection: keep-alive
6,,X-Powered-By: KPHP/7.4.116528
7,,"Set-Cookie: remixir=DELETED; expires=Thu, 01 Jan 1970 00:00:01 GMT; path=/; domain=.vk.com; secure; HttpOnly"
8,,Cache-control: no-store
9,,X-Frame-Options: DENY


In [13]:
# Выполнить загрузку файла
import subprocess
import json
server_url = resp.json()['response']['upload_url']
cmd = ["curl", "-F", 'file=@/tmp/galtse.cx.txt', server_url]
print('----> ', cmd)
text = subprocess.check_output(cmd)
upload_data = json.loads(text)

---->  ['curl', '-F', 'file=@/tmp/galtse.cx.txt', 'https://pu.vk.com/c909218/upload_doc.php?act=add_doc&mid=450410456&aid=0&gid=0&type=0&hash=dcdecbbc5f212ad0fa2e46303ee6d3be&rhash=f0a454129abb8d8a946dd13195b597ea&api=1']


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

100  180k  100   157  100  179k    305   349k --:--:-- --:--:-- --:--:--  351k


In [14]:
# Финализировать сохранение документа
print_transaction(
    requests.post("https://api.vk.com/method/docs.save",
                  params={
                      "access_token": USER_TOKEN,
                      "v": "5.199",
                      "file": upload_data['file'],
                      "title": "2 Johns 1 Galt [GONE WRONG] explicit.txt"
                  })
)

,0,1
0,POST https://api.vk.com/method/docs.save?access_token=**USER_TOKEN**&v=5.199&file=450410456%7C0%7C0%7C909218%7C59107e852a%7Ctxt%7C184029%7Cgaltse.cx.txt%7C5bb75515ea4305cc70f0ca64772b95b7%7C26d60395bb8263e094d49c199d9b0f16%7C%7C%7C%7CeyJkaXNrIjoiMjMifQ%3D%3D&title=2+Johns+1+Galt+%5BGONE+WRONG%5D+explicit.txt,200
1,User-Agent: python-requests/2.31.0,Server: kittenx
2,"Accept-Encoding: gzip, deflate, br","Date: Sat, 20 Apr 2024 13:35:19 GMT"
3,Accept: */*,Content-Type: application/json; charset=utf-8
4,Connection: keep-alive,Content-Length: 318
5,Content-Length: 0,Connection: keep-alive
6,,X-Powered-By: KPHP/7.4.116528
7,,"Set-Cookie: remixir=DELETED; expires=Thu, 01 Jan 1970 00:00:01 GMT; path=/; domain=.vk.com; secure; HttpOnly"
8,,Cache-control: no-store
9,,X-Frame-Options: DENY


## Фотографии

Методы `photos` позволяют работать с фотографиями.
Для них также требуется отдельный scope,
который указан в URL запроса выше.

Чтобы получить фотографии какого-то пользователя или сообщества, можно использовать метод `photos.getAll`.
Каждая фотография доступна в нескольких размерах, список которых можно получить с помощью параметра `photo_sizes`;
определить, является ли фотография скрытой, можно с `need_hidden`.

In [15]:
print_transaction(
    requests.post("https://api.vk.com/method/photos.getAll",
                  params={
                      "access_token": USER_TOKEN,
                      "v": "5.199",
                      "owner_id": -403392,
                      "count": 5,
                      "photo_sizes": 1,
                      "need_hidden": 1
                  })
)

,0,1
0,POST https://api.vk.com/method/photos.getAll?access_token=**USER_TOKEN**&v=5.199&owner_id=-403392&count=5&photo_sizes=1&need_hidden=1,200
1,User-Agent: python-requests/2.31.0,Server: kittenx
2,"Accept-Encoding: gzip, deflate, br","Date: Sat, 20 Apr 2024 13:35:21 GMT"
3,Accept: */*,Content-Type: application/json; charset=utf-8
4,Connection: keep-alive,Content-Length: 3915
5,Content-Length: 0,Connection: keep-alive
6,,X-Powered-By: KPHP/7.4.116528
7,,"Set-Cookie: remixir=DELETED; expires=Thu, 01 Jan 1970 00:00:01 GMT; path=/; domain=.vk.com; secure; HttpOnly"
8,,Cache-control: no-store
9,,X-Frame-Options: DENY


Чтобы получить комментарии к фотографии, можно использовать `photos.getComments`.

In [16]:
print_transaction(
    requests.post("https://api.vk.com/method/photos.getComments",
                  params={
                      "access_token": USER_TOKEN,
                      "v": "5.199",
                      "owner_id": -403392,
                      "photo_id": 456239362,
                      "need_likes": 1
                  })
)

,0,1
0,POST https://api.vk.com/method/photos.getComments?access_token=**USER_TOKEN**&v=5.199&owner_id=-403392&photo_id=456239362&need_likes=1,200
1,User-Agent: python-requests/2.31.0,Server: kittenx
2,"Accept-Encoding: gzip, deflate, br","Date: Sat, 20 Apr 2024 13:35:22 GMT"
3,Accept: */*,Content-Type: application/json; charset=utf-8
4,Connection: keep-alive,Content-Length: 190
5,Content-Length: 0,Connection: keep-alive
6,,X-Powered-By: KPHP/7.4.116528
7,,"Set-Cookie: remixir=DELETED; expires=Thu, 01 Jan 1970 00:00:01 GMT; path=/; domain=.vk.com; secure; HttpOnly"
8,,Cache-control: no-store
9,,X-Frame-Options: DENY


Загрузка фотографий в целом похожа на загрузку документов. Главное отличие -- здесь больше методов `get*UploadServer`,
а также несколько методов `save` в зависимости от того, куда загружается фотография;
также, для того, чтобы фотография стала доступна пользователям,
нужно создать сущность (например, пост на стене),
которая будет ссылаться на нее.
Например, чтобы загрузить фотографию на стену текущего пользователя, нужно сделать следующее:

In [17]:
# Получить фотографию
with open('/tmp/big-tits.jpg', 'wb') as o:
    o.write(requests.get('https://upload.wikimedia.org/wikipedia/commons/e/e0/Great_tit_feeding.JPG', headers={'user-agent': 'rudn-year3-api-lab/3'}).content)

In [18]:
# Получить адрес для загрузки
resp = requests.post("https://api.vk.com/method/photos.getWallUploadServer",
                  params={
                      "access_token": USER_TOKEN,
                      "v": "5.199",
                  })

print_transaction(resp)

,0,1
0,POST https://api.vk.com/method/photos.getWallUploadServer?access_token=**USER_TOKEN**&v=5.199,200
1,User-Agent: python-requests/2.31.0,Server: kittenx
2,"Accept-Encoding: gzip, deflate, br","Date: Sat, 20 Apr 2024 13:35:24 GMT"
3,Accept: */*,Content-Type: application/json; charset=utf-8
4,Connection: keep-alive,Content-Length: 218
5,Content-Length: 0,Connection: keep-alive
6,,X-Powered-By: KPHP/7.4.116528
7,,"Set-Cookie: remixir=DELETED; expires=Thu, 01 Jan 1970 00:00:01 GMT; path=/; domain=.vk.com; secure; HttpOnly"
8,,Cache-control: no-store
9,,X-Frame-Options: DENY


In [19]:
# Выполнить загрузку
import subprocess
import json
server_url = resp.json()['response']['upload_url']
cmd = ["curl", "-F", 'file=@/tmp/big-tits.jpg', server_url]
print('----> ', cmd)
text = subprocess.check_output(cmd)
upload_data = json.loads(text)

---->  ['curl', '-F', 'file=@/tmp/big-tits.jpg', 'https://pu.vk.com/c221232/ss2130/upload.php?act=do_add&mid=450410456&aid=-14&gid=0&hash=3daac14360ca1ca264ed3a261cb9d850&rhash=9ea0bb997ef779a447709d2bb9bb48f3&swfupload=1&api=1&wallphoto=1']


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

100 4813k  100  1855  100 4811k   1498  3886k  0:00:01  0:00:01 --:--:-- 3887k


In [20]:
# Финализировать сохранение фотографии
resp = requests.post("https://api.vk.com/method/photos.saveWallPhoto",
                  params={
                      "access_token": USER_TOKEN,
                      "v": "5.199",
                      "photo": upload_data['photo'],
                      "server": upload_data['server'],
                      "hash": upload_data['hash'],
                      "caption": "feeding two GREAT big TITS (\U0001f51e)"
                  })

print_transaction(resp)

,0,1
0,POST https://api.vk.com/method/photos.saveWallPhoto?access_token=**USER_TOKEN**&v=5.199&photo=%5B%7B%22markers_restarted%22%3Atrue%2C%22photo%22%3A%222dc8682769%3Aw%22%2C%22sizes%22%3A%5B%5D%2C%22latitude%22%3A0%2C%22longitude%22%3A0%2C%22kid%22%3A%22426ef86defa867b8c08732a29737108b%22%2C%22sizes2%22%3A%5B%5B%22s%22%2C%2200a3b5eb83fe3094a1bc189fbf46e04eff6496fc47c2fdb481b0e2ea%22%2C%22-5917358728068311386%22%2C75%2C33%5D%2C%5B%22m%22%2C%22f7c75b50c98bd061147bba64a708a84ce2ac5dbd6aa2fb6f967717e9%22%2C%22725351612944968393%22%2C130%2C57%5D%2C%5B%22x%22%2C%229c37d96a744e58055bd05feed181c13c28d76f7de439a899b356f8f0%22%2C%22-2134583644479094607%22%2C604%2C264%5D%2C%5B%22y%22%2C%22f6784f7da012f4bfea34cb140f2b26e3c2a4938bfc9013c5968214b7%22%2C%228679319103956896396%22%2C807%2C353%5D%2C%5B%22z%22%2C%226dbcdb6c3d6d12d36163e31f602e8d576b4b49a18581709a3654efbf%22%2C%224793788529259416761%22%2C1280%2C560%5D%2C%5B%22w%22%2C%22dab0964acd87c3c67ed70844f96faa328ad188a992d5c85162ef90a2%22%2C%22-2428153992372671426%22%2C2560%2C1120%5D%2C%5B%22o%22%2C%228e8fd27c7d97dbe54ec9bb6584045500c2fae5d7217da61aa5b83011%22%2C%228337602749479996383%22%2C130%2C87%5D%2C%5B%22p%22%2C%2209fd089fc88c6b950419c7d8f2632825045e4d5d039bd286e3673d2e%22%2C%22-3603014852473863366%22%2C200%2C133%5D%2C%5B%22q%22%2C%2208a707a3b477f9d276f92224264f5b69a7a1f1bba60dbd596f97e7c6%22%2C%223600790417852304878%22%2C320%2C213%5D%2C%5B%22r%22%2C%226af98b602eb662de43476ef0aa8bca37e9cb1beadadc0df69e1cf72e%22%2C%226711826619350478853%22%2C510%2C340%5D%5D%2C%22urls%22%3A%5B%5D%2C%22urls2%22%3A%5B%22AKO164P-MJShvBifv0bgTv9klvxHwv20gbDi6g%2FplbunJZR4a0.jpg%22%2C%2298dbUMmL0GEUe7pkpwioTOKsXb1qovtvlncX6Q%2FyWrX6nL3EAo.jpg%22%2C%22nDfZanROWAVb0F_u0YHBPCjXb33kOaiZs1b48A%2FsQB3H3xvYOI.jpg%22%2C%229nhPfaAS9L_qNMsUDysm48Kkk4v8kBPFloIUtw%2FjMqcc70ic3g.jpg%22%2C%22bbzbbD1tEtNhY-MfYC6NV2tLSaGFgXCaNlTvvw%2FuYzZzkT1hkI.jpg%22%2C%222rCWSs2Hw8Z-1whE-W-qMorRiKmS1chRYu-Qog%2FPiyuEM92Td4.jpg%22%2C%22jo_SfH2X2-VOybtlhARVAML65dchfaYapbgwEQ%2F30sTsYYdtXM.jpg%22%2C%22Cf0In8iMa5UEGcfY8mMoJQReTV0Dm9KG42c9Lg%2FOsPWzRSF_80.jpg%22%2C%22CKcHo7R3-dJ2-SIkJk9baaeh8bumDb1Zb5fnxg%2F7o23ps6T-DE.jpg%22%2C%22avmLYC62Yt5DR27wqovKN-nLG-ra3A32nhz3Lg%2FBXBl4L8yJV0.jpg%22%5D%7D%5D&server=221232&hash=70ca8f078cabbfd0817a8495da8d3c03&caption=feeding+two+GREAT+big+TITS+%28%F0%9F%94%9E%29,200
1,User-Agent: python-requests/2.31.0,Server: kittenx
2,"Accept-Encoding: gzip, deflate, br","Date: Sat, 20 Apr 2024 13:35:26 GMT"
3,Accept: */*,Content-Type: application/json; charset=utf-8
4,Connection: keep-alive,Content-Length: 1164
5,Content-Length: 0,Connection: keep-alive
6,,X-Powered-By: KPHP/7.4.116528
7,,"Set-Cookie: remixir=DELETED; expires=Thu, 01 Jan 1970 00:00:01 GMT; path=/; domain=.vk.com; secure; HttpOnly"
8,,Cache-control: no-store
9,,X-Frame-Options: DENY


In [21]:
# Опубликовать пост на стене, который будет содержать эту фотографию
resp_data = resp.json()['response'][0]
print_transaction(
    requests.post("https://api.vk.com/method/wall.post",
                  params={
                      "access_token": USER_TOKEN,
                      "v": "5.199",
                      "attachments": f'photo{resp_data["owner_id"]}_{resp_data["id"]}',
                  })
)

,0,1
0,POST https://api.vk.com/method/wall.post?access_token=**USER_TOKEN**&v=5.199&attachments=photo450410456_457283112,200
1,User-Agent: python-requests/2.31.0,Server: kittenx
2,"Accept-Encoding: gzip, deflate, br","Date: Sat, 20 Apr 2024 13:35:28 GMT"
3,Accept: */*,Content-Type: application/json; charset=utf-8
4,Connection: keep-alive,Content-Length: 50
5,Content-Length: 0,Connection: keep-alive
6,,X-Powered-By: KPHP/7.4.116528
7,,"Set-Cookie: remixir=DELETED; expires=Thu, 01 Jan 1970 00:00:01 GMT; path=/; domain=.vk.com; secure; HttpOnly, remixstlid=9098286578633658923_LVL88W5zjg57TFjZ5Y9dIfQzK0MNvIYNHKdBzZmYrzw; expires=Sun, 20 Apr 2025 13:35:27 GMT; path=/; domain=.vk.com; secure"
8,,Cache-control: no-store
9,,X-Frame-Options: DENY


## Заключение

При взаимодействии с сложными системами, вроде VK, API также будут иметь довольно большую сложность.
Если документация написана хорошо, то разработчики могут понять эту сложность
и написать код, который будет успешно взаимодействовать с этим API.

В следующей лабораторной работе мы рассмотрим, как сделать это более удобным с помощью JSON Schema, которые можно использовать для упрощения таких задач, например для автоматической генерации документации и кода, который создает запросы и парсит ответы.